In [8]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model

# Path to the train2017 folder
train_dir = "train"

# Function to extract aesthetic features
def extract_aesthetic_features(image):
    # Dullness: Average brightness
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dullness = np.mean(gray)
    
    # Whiteness: Average intensity of white pixels
    whiteness = np.mean(image[image > 200]) if np.any(image > 200) else 0
    
    # Uniformity: Standard deviation of pixel values
    uniformity = np.std(image)
    
    # Colors: Dominant and average colors
    average_color = np.mean(image, axis=(0, 1))
    reshaped_image = image.reshape(-1, 3)
    unique_colors, counts = np.unique(reshaped_image, axis=0, return_counts=True)
    dominant_color = unique_colors[np.argmax(counts)]
    
    # Dimensions: Width, height, and size
    height, width, _ = image.shape
    size = height * width
    
    # Blurriness: Variance of Laplacian
    gray_laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    blurriness = gray_laplacian.var()
    
    return [
        dullness,
        whiteness,
        uniformity,
        average_color[0],  # Red channel
        average_color[1],  # Green channel
        average_color[2],  # Blue channel
        dominant_color[0],  # Red
        dominant_color[1],  # Green
        dominant_color[2],  # Blue
        width,
        height,
        size,
        blurriness
    ]

In [9]:
# Prepare dataset: Images and extracted features
def prepare_dataset(image_dir, img_size=(128, 128)):
    images = []
    features = []
    
    for img_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue
        
        # Resize image for CNN
        resized_image = cv2.resize(image, img_size)
        images.append(resized_image / 255.0)  # Normalize to [0, 1]
        
        # Extract features
        feature_vector = extract_aesthetic_features(image)
        features.append(feature_vector)
    
    return np.array(images), np.array(features)

# Load dataset
images, features = prepare_dataset(train_dir)

# Generate dummy MOS scores (replace this with real MOS scores if available)
mos_scores = np.random.uniform(1, 10, size=len(images))

In [10]:
# CNN model for image features
input_image = Input(shape=(128, 128, 3))
x = Conv2D(32, (3, 3), activation='relu')(input_image)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
cnn_output = Dense(128, activation='relu')(x)

# Input for extracted features
input_features = Input(shape=(13,))
combined = Concatenate()([cnn_output, input_features])

# Output layer for MOS prediction
output = Dense(1, activation='linear')(combined)

In [11]:
model = Model(inputs=[input_image, input_features], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit([images, features], mos_scores, epochs=10, batch_size=32)

Epoch 1/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 53s 191ms/step - loss: 704057920.0000 - mae: 19597.3398
Epoch 2/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 47s 180ms/step - loss: 41762272.0000 - mae: 4922.1372
Epoch 3/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 47s 181ms/step - loss: 20937354.0000 - mae: 3427.2539
Epoch 4/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 47s 181ms/step - loss: 10232039.0000 - mae: 2347.0127
Epoch 5/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 48s 184ms/step - loss: 4357681.5000 - mae: 1534.5621
Epoch 6/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 49s 187ms/step - loss: 1733621.6250 - mae: 967.4663
Epoch 7/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 50s 191ms/step - loss: 689842.4375 - mae: 611.1973
Epoch 8/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 57s 220ms/step - loss: 278600.5000 - mae: 390.8602
Epoch 9/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 56s 216ms/step - loss: 124188.1172 - mae: 262.7506
Epoch 10/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 53s 203ms/step - loss: 59714.4141 - mae: 182.9204


In [12]:
model.save("mos_prediction_model.h5")
